In [1]:
import speech_recognition as sr
import pyttsx3 
from translate import Translator
from gtts import gTTS
from playsound import playsound
import json
import numpy as np 
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import colorama
colorama.init
from colorama import Fore, Style, Back
import random
import pickle
import os
from pathlib import Path


In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
df = pd.read_csv('books.csv', error_bad_lines = False)

C:\Users\safee\AppData\Local\Temp\ipykernel_7584\397317604.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines = False)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [4]:
# converting average rating column into categorical column
def num_into_obj(x):
    if x>=0 and x<=1:
        return 'between 0 and 1'
    elif x>1 and x<=2:
        return 'between 1 and 2'
    elif x>2 and x<=3:
        return 'between 2 and 3'
    elif x>3 and x<=4:
        return 'between 3 and 4'
    else:
        return 'between 4 and 5'
    
df['rating_obj'] = df['average_rating'].apply(num_into_obj)

In [5]:
# Let's encode the categorical column
rating_df = pd.get_dummies(df['rating_obj'])
#rating_df.head()

In [6]:
# Let's encode the language code column as well
language_df = pd.get_dummies(df['language_code'])
#language_df.head()

In [7]:
# Let's concat both the data frames and set the title column as the index 
features = pd.concat([rating_df, language_df, df['average_rating'], df['ratings_count'],df['title']], axis=1)
features.set_index('title', inplace=True)
#features.head()

In [8]:
min_max_scaler = MinMaxScaler()
features_scaled = min_max_scaler.fit_transform(features)

In [9]:
model = neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree', metric='euclidean')
model.fit(features_scaled)
dist, idlist = model.kneighbors(features_scaled)

In [10]:
# Initialize the recognizer 
r = sr.Recognizer() 

In [57]:
# Function to convert text to
# speech
def SpeakText(command):
      
    # Initialize the engine
    engine = pyttsx3.init()
    engine.say(command) 
    engine.runAndWait()

def recommend_books_on_authors(authors):
    a = df[df['authors']==authors][['title','average_rating']]
    a = a.sort_values(by = 'average_rating', ascending=False)
    h = a.head(6)
    for x in h['title']:
        SpeakText(x)
        print(x)
        
def recommend_books_on_publishers(publishers): 
    a = df[df['publisher']==publishers][['title','average_rating']]
    a = a.sort_values(by = 'average_rating', ascending=False)
    h = a.head(6)
    for x in h['title']:
        SpeakText(x)
        print(x)       
        
def BookRecommender(book):
    book_list_name = []
    book_id = df[df['title'] == book].index
    book_id = book_id[0]
    #if MyText.lower() == "thank you":
    try:        
        #break
        for newid in idlist[book_id]:
            book_list_name.append(df.loc[newid].title)
            SpeakText(df.loc[newid].title)
            print(df.loc[newid].title)
        return
    except Exception as err:
        print( "NUBI:I can't hear you")

In [55]:
# Loop infinitely for user to
# speak
model = keras.models.load_model('voice_assistant')
with open('tokenizer.pickle','rb') as handle:
    tokenizer = pickle.load(handle)
with open('lbl_encoder.pickle','rb') as ecn:
    lbl_encoder = pickle.load(ecn)
max_len = 20
count=104
while(1):    
      
    # Exception handling to handle
    # exceptions at the runtime
    try:
          
        # use the microphone as source for input.
        with sr.Microphone() as source2:
              
            # wait for a second to let the recognizer
            # adjust the energy threshold based on
            # the surrounding noise level 
            r.adjust_for_ambient_noise(source2, duration=0.2)
              
            #listens for the user's input 
            audio2 = r.listen(source2)
              
            # Using ggogle to recognize audio
            MyText = r.recognize_google(audio2)
            MyText = MyText.lower()
            translator=Translator(from_lang='en',to_lang="ml")
            
            
            
            
            
  

            print("User: "+ MyText)
            #print(Fore.LIGHTBLUE_EX + "User: "+ Style.RESET_ALL,end="")
            #inp = input()
            if MyText.lower() == "bye":
                break
            if MyText.lower() == "book search":
                text="You can search for a book in three ways: by author, publisher, and title. Which one is your preferred choice? "
                translation=translator.translate(text)
                print(Fore.GREEN + "NUBI: "+translation)
                mal=gTTS(translation,lang="ml")
                mal.save("voice5.mp3")
                playsound("voice5.mp3")
                os.remove("voice5.mp3")
            
                #print("Tell book name you like")
                #SpeakText("Tell book name you like")
                while(1):
                    try:
                        with sr.Microphone() as source2:
                            r.adjust_for_ambient_noise(source2, duration=0.2)
                            audio2 = r.listen(source2)
                            MyBook = r.recognize_google(audio2)
                            MyBooks = MyBook.lower()
                            book=MyBooks.title()
                            print(book)
                            if MyBook.lower() == "author" or MyBook.lower() == "other" or MyBook.lower() == "order" or MyBook.lower() == "weather":
                                text="tell your author name "
                                translation=translator.translate(text)
                                print(Fore.GREEN + "NUBI: "+translation)
                                mal=gTTS(translation,lang="ml")
                                mal.save("voice5.mp3")
                                playsound("voice5.mp3")
                                os.remove("voice5.mp3")
                                with sr.Microphone() as source3:
                                     r.adjust_for_ambient_noise(source3, duration=0.2)
                                     audio3 = r.listen(source3)
                                     author = r.recognize_google(audio3)
                                     authos = author.lower()
                                     authors=authos.title()
                                     print( authors)
                            if MyBook.lower() == "title" :
                                text="tell your book name "
                                translation=translator.translate(text)
                                print(Fore.GREEN + "NUBI: "+translation)
                                mal=gTTS(translation,lang="ml")
                                mal.save("voice5.mp3")
                                playsound("voice5.mp3")
                                os.remove("voice5.mp3")
                                with sr.Microphone() as source3:
                                     r.adjust_for_ambient_noise(source3, duration=0.2)
                                     audio3 = r.listen(source3)
                                     bookk = r.recognize_google(audio3)
                                     boookk = bookk.lower()
                                     bookks=boookk.title()
                                     print(bookks)
                            if MyBook.lower() == "publisher" :
                                text="tell your publisher name "
                                translation=translator.translate(text)
                                print(Fore.GREEN + "NUBI: "+translation)
                                mal=gTTS(translation,lang="ml")
                                mal.save("voice5.mp3")
                                playsound("voice5.mp3")
                                os.remove("voice5.mp3")
                                with sr.Microphone() as source3:
                                     r.adjust_for_ambient_noise(source3, duration=0.2)
                                     audio3 = r.listen(source3)
                                     publisher = r.recognize_google(audio3)
                                     publishers = publisher.lower()
                                     publisherss=publishers.title()
                                     print(publisherss)
                                     
                                     

                            print(Fore.GREEN + "NUBI: "+"I would prefer that you read.")
                            SpeakText("I would prefer that you read")
                            if MyBook.lower() == "author" or MyBook.lower() == "other" or MyBook.lower() == "order" or MyBook.lower() == "weather":
                                recommend_books_on_authors("Crown")
                            if MyBook.lower() == "title" :
                                BookRecommender(bookks)
                            if MyBook.lower() == "publisher" :
                                recommend_books_on_publishers("Bill Bryson")
                                
                            print( authors)
                    except sr.RequestError as e:
                        print("Could not request results; {0}".format(e))
                        
            result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([MyText]),
                                                                           truncating='post',maxlen=max_len))
        
            tag = lbl_encoder.inverse_transform([np.argmax(result)])
            for i in data['intents']:
                if i['tag']==tag:
                    print(Fore.GREEN + "NUBI: "+ Style.RESET_ALL,np.random.choice(i['responses']))
                    translation=translator.translate(np.random.choice(i['responses']))
                    print("NUBI:"+translation)
                    mal=gTTS(translation,lang="ml")
                    #mal.save("voice"+chr(count)+".mp3")
                    mal.save("voice.mp3")
                    #audio_file="voice"+chr(count)+".mp3"
                    #audio = Path().cwd() / audio_file
                    playsound("voice.mp3")
                    os.remove("voice.mp3")
                    count=count+1
                    #print("User: "+np.random.choice(i['responses']))
                    #SpeakText(np.random.choice(i['responses']))     
            
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))
          
    except sr.UnknownValueError:
        translator=Translator(from_lang='en',to_lang="ml")
        text="I'm sorry, but I'm unable to hear you."
        translation=translator.translate(text)
        print(Fore.GREEN + "chatBot: "+translation)
        mal=gTTS(translation,lang="ml")
        mal.save("voice2.mp3")
        playsound("voice2.mp3")
        #audio_file="voice2.mp3"
        #audio = Path().cwd() / audio_file
        os.remove("voice2.mp3")
        #SpeakText("I can't hear you")

User: book search
NUBI: നിങ്ങൾക്ക് മൂന്ന് തരത്തിൽ ഒരു പുസ്തകം തിരയാൻ കഴിയും: രചയിതാവ്, പ്രസാധകൻ, പേര് എന്നിവ പ്രകാരം. നിങ്ങളുടെ ഇഷ്ടപ്പെട്ട ചോയ്‌സ് ഏതാണ്?
Publisher
NUBI: നിങ്ങളുടെ പ്രസാധകന്റെ പേര് പറയുക
Building On
NUBI: I would prefer that you read.
Stephen
chatBot: എന്നോട് ക്ഷമിക്കൂ, പക്ഷേ എനിക്ക് നിങ്ങളുടെ വാക്കുകൾ കേൾക്കാൻ കഴിയുന്നില്ല.
chatBot: എന്നോട് ക്ഷമിക്കൂ, പക്ഷേ എനിക്ക് നിങ്ങളുടെ വാക്കുകൾ കേൾക്കാൻ കഴിയുന്നില്ല.
User: bye


In [32]:

def recommend_books_on_publishers(x):
    a = df[df['publisher']==x][['title','average_rating']]
    a = a.sort_values(by = 'average_rating', ascending=False)
    b = a.head(10)
    return b['title']

In [33]:
 recommend_books_on_publishers('Crown')

9        The Hitchhiker's Guide to the Galaxy (Hitchhik...
10951                         The Unpublished David Ogilvy
2444        World War Z: An Oral History of the Zombie War
2795     Dreams from My Father: A Story of Race and Inh...
2143                                     The Metamorphosis
2652     The Audacity of Hope: Thoughts on Reclaiming t...
6783                       The Little Baby Snoogle-Fleejer
Name: title, dtype: object

In [ ]:

playsound("voiceh.mp3")

In [21]:
recommend_books_on_publishers('Crown')

IndentationError: unexpected indent (1061942205.py, line 2)

In [51]:
   def recommend_books_on_publisherss(authors):       
         a = df[df['authors']==authors][['title','average_rating']]
         a = a.sort_values(by = 'average_rating', ascending=False)
         h = a.head(6)
         for x in h['title']:        
            SpeakText(x)
            print(x)
            

In [56]:
recommend_books_on_publishers('Bill Bryson')